# Lecture 4 - Robustness of DNNs

On Assignment 1, we saw that DNNs are very good at learning some tasks such as MNIST digit classification. At the same time, we saw the impact of data bias towards the output classification. Here, we will see an inherent flaw in the way we do inference on the trained models.

In [ ]:
import os
import time
import torch
import itertools
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

flatten = itertools.chain.from_iterable

# Some helper functions

def plot_loss(loss_as_list):
    """
    Plot the loss curve from a list of loss terms.
    """
    plt.plot(loss_as_list, 'k')
    _ = plt.title("Loss Curve")
    _ = plt.xlabel("Epochs")
    _ = plt.ylabel("Loss")
    
def get_classification_results(model, loader):
    """
    Print the accuracy of a trained model.
    Loss: Cross Entropy
    """
    correct, total = 0, 0
    predictions = []
    true_labels = []

    for xs, ts in test_loader:
        xs = xs.view(-1, 784) # flatten the image
        zs = model(xs) # do forward pass
        pred = zs.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(ts.view_as(pred)).sum().item() # count equal values
        total += int(ts.shape[0]) # get total values

        predictions.append(pred)
        true_labels.append(ts)

    accuracy = correct / total
    conf_matrix = confusion_matrix(list(flatten(true_labels)), list(flatten(predictions)))
    cl_report = classification_report(list(flatten(true_labels)), list(flatten(predictions)), digits=4)

    print(cl_report)
    print(conf_matrix)

### Load original MNIST data

In [ ]:
torch.manual_seed(13)

N_train = 64
N_test = 256

# We will use torch.utils.data.DataLoader to wrap our dataset.
# This provides easier batching, GPU support, etc.
# Calling torchvision.datasets.MNIST() will download and format the MNIST
# dataset with the transforms we specify. Here, in the transforms we first convert
# the image to PyTorch tensor, and then normalize the image based on a given mean
# and standard deviation. Normalizing the image does: image = (image - mean) / std.
# We shuffle the data as well by defining shuffle=True.

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../Datasets/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=N_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../Datasets/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=N_test, shuffle=True)

### Define the model and hyperparameters

In [ ]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Hyper Parameters
lr = 0.003 # learning rate
NUM_EPOCHS = 10

In [ ]:
def MLP():
    """
    A function implementation of the model definition.
    """
    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size)
                     )
    return model

In [ ]:
def train(model, NUM_EPOCHS, train_loader):
    """
    A function to train the neural network model.
    """
    loss_fn = nn.CrossEntropyLoss() # also called criterion sometimes.

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    start = time.time()
    loss_as_list = []

    for EPOCH in range(NUM_EPOCHS):
        running_loss = 0
        for images, labels in train_loader:
            # Flatten MNIST images into a 784 long vector
            images = images.view(images.shape[0], -1)

            # Training pass
            optimizer.zero_grad()

            output = model(images)
            loss = loss_fn(output, labels)
            loss_as_list.append(loss)

            #This is where the model learns by backpropagating
            loss.backward()

            #And optimizes its weights here
            optimizer.step()

            running_loss += loss.item()
        else:
            print("Epoch {} - Training loss: {}".format(EPOCH, running_loss/len(train_loader)))

    print("\nTraining Time (in minutes) =",(time.time()-start)/60)
    return(loss_as_list)

### Load the trained DNN model

We shall try to load the trained DNN on the MNIST data from Assignment 1. If the file doesn't exist, we will train the network again.

In [ ]:
if os.path.exists("../Assignment-1/mnist_original.pt"):
    model = torch.load("../Assignment-1/mnist_original.pt")
else:
    # Define the model
    model = MLP()

    # Start training the model on the train_loader.
    loss_values = train(model, NUM_EPOCHS, train_loader)
    
    plot_loss(loss_values)
    
    get_classification_results(model, test_loader)
    
    # Save the model
    torch.save(model, 'mnist_original.pt')

In [ ]:
# Study the representations

# Extract the hidden layer latent dense representations to study the distribution of the representations.
# We can select a specific layer for our study.
# Let us extract the penultimate layer as the embedding layer. Hence we use `-2` index since `:` means upto but not including.

embd_model = nn.Sequential(*list(model.children()))[:-2]

In [ ]:
# Note that our new model don't have the output layer and associated ReLU activation layer.
embd_model

In [ ]:
# We can define a z vector and append all activations to that.
zs = []
for xs, ts in test_loader:
    xs = xs.view(-1, 784) # flatten the image
    zs.append(embd_model(xs).detach().numpy()) # do forward pass to extract embeddings and append to zs.

zs = np.vstack(zs) # Stack all the embeddings. This will give you 10000*64 array, since embedding size (out_features of (2) Linear) is 64.

zs_mean = np.average(zs, axis=0) # Find mean of all embeddings with respect to depth axis (axis=0). This will give you 1*64 vector.

zs_std = np.std(zs, axis=0) # Find the standard deviation all embeddings with respect to the depth axis.

### Visualize some resutls

In [ ]:
test_subset = enumerate(test_loader)
batch_idx, (one_batch_of_test_subset_x, one_batch_of_test_subset_y) = next(test_subset)

In [ ]:
i = 0
plt.imshow(one_batch_of_test_subset_x[i][0], cmap='gray', interpolation='none')
_ = plt.title("Ground Truth: {}".format(one_batch_of_test_subset_y[i]))

In [ ]:
# Extract the output logit

output_logits = model(one_batch_of_test_subset_x[i][0].view(-1, 784))
print(output_logits)

In [ ]:
# If required you can convert the tensor to a numpy array by calling detach() and numpy()
output_logits.detach().numpy()

In [ ]:
# Find the maximum logit value
output_logits.max(1, keepdim=True)

We see that the maximum value and it's location corresponds well to output position we were looking for! How can we extract this using code?

In [ ]:
pred = output_logits.max(1, keepdim=True)[1]
print("Predicted Label:", pred)

In [ ]:
true_label = one_batch_of_test_subset_y[i].view_as(pred).sum().item()
print("True Label:", true_label)

Great! We have a way to do model inference on new inputs.

### Feed random gaussian noise as input

Instead of feeding an actual MNIST image, what happens if we feed a noisy image, randomly sampled from a gaussian distribution? Would our model know whether it is just noise that we are feeding it? Would it complain that it cannot find a proper answer? Let's find out...

In [ ]:
# Sample a gaussian noise vector from a normal distribution
mu, sigma = 0, 0.2
gaussian_noise = np.random.normal(mu, sigma, 784)

# Plot the noise vector as an image
plt.imshow(gaussian_noise.reshape(28,28), cmap='gray', interpolation='none')
_ = plt.title("Ground Truth: NONE")

# Convert the noise vector to a tensor
gaussian_noise = torch.tensor(gaussian_noise, dtype=torch.float)

In [ ]:
output_logits_noise = model(gaussian_noise.view(-1, 784))
print(output_logits_noise)

In [ ]:
pred_noise = output_logits_noise.max(1, keepdim=True)[1]
print("Predicted Label:", pred_noise)

The model we trained predicted a $3$ for the gaussian noise. Isn't this an unfavorable outcome? What if we were to use this model in a scenario where the decisions are mission-critical. We need to think about such out-of-distribution data as we move forward!

### How different are the activations?

We can extract the embeddings using the same method we used in Assignment 1. Let's extract the embedding for the gaussian noise input and compare with the mean of the clean embeddings we found earlier in the notebook.

In [ ]:
zs_noise = embd_model(gaussian_noise)

In [ ]:
plt.plot(zs_mean, 'k', label='Mean of clean embeddings')
plt.plot(zs_noise.detach().numpy(), 'r', label='Noise Input')
plt.xlabel('Element location within embedding')
plt.ylabel('Embedding value')
plt.legend()

plt.show()

What is your observation?